In [1]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
from pathlib import Path
import datetime

logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", 
                    level=logging.INFO)

In [2]:
OUTPUT_DATA = Path("data")
OUTPUT_PATH = Path("outputs")

# Reading DataFrame

In [3]:
dfs_load_energy = []
for path_file_csv in glob.glob(os.path.join(OUTPUT_DATA, "*.csv")):
    logging.info(f"Loading DataFrame from the file {path_file_csv}...")
    dfs_load_energy.append(pd.read_csv(path_file_csv, sep=";", encoding="utf-8"))

2023-05-17 11:54:28,656 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2012.csv...
2023-05-17 11:54:28,659 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2021.csv...
2023-05-17 11:54:28,662 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2000.csv...
2023-05-17 11:54:28,664 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2002.csv...
2023-05-17 11:54:28,666 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2017.csv...
2023-05-17 11:54:28,668 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2011.csv...
2023-05-17 11:54:28,670 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2023.csv...
2023-05-17 11:54:28,672 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2006.csv...
2023-05-17 11:54:28,674 - root - INFO - Loading DataFrame from the file data/CARGA_ENERGIA_2003.csv...
2023-05-17 11:54:28,675 - root - INFO - Loading DataFrame from the file d

In [4]:
df_load_energy = pd.concat(dfs_load_energy, ignore_index=True)

In [5]:
from utils.dataframe_tools import create_pivot_table_load_energy

df_load_energy_one_column = create_pivot_table_load_energy(df_load_energy)
df_load_energy_one_column.index = pd.DatetimeIndex(df_load_energy_one_column.index)
df_load_energy_one_column.sort_index(ascending=True, inplace=True)

## Checking data range if is correct

In [6]:
from utils.dataframe_tools import check_date_range_energy_load

In [7]:
start_date = df_load_energy_one_column.index.min()
end_date = df_load_energy_one_column.index.max()
start_date, end_date

(Timestamp('2000-01-01 00:00:00'), Timestamp('2023-05-14 00:00:00'))

In [8]:
check_date_range_energy_load(df_load_energy_one_column.index, start_date, end_date)

ValueError: Failed to compare date ranges. 8523 != 8535 

There're some data missing on the original datasets (https://dados.ons.org.br/dataset/carga-energia), I'll fill with NaN values and substitute their after.

In [9]:
idx = pd.date_range(start_date, end_date, freq="D")
df_load_energy_one_column = df_load_energy_one_column.reindex(idx, fill_value=np.nan) 
df_load_energy_one_column.index.name = "din_instante"

# Preprocessing

In [10]:
from utils.dataframe_tools import data_clean, replace_zero_negative

df_load_energy_one_column = data_clean(df_load_energy_one_column)

2023-05-17 11:54:36,336 - root - INFO - Number of NAs values = 48
2023-05-17 11:54:36,337 - root - INFO - Replacing NAs values with the next value...
2023-05-17 11:54:36,338 - root - INFO - Number of negative values in dataframe = 0
2023-05-17 11:54:36,340 - root - INFO - Number of zero values in dataframe = 0
2023-05-17 11:54:36,343 - root - INFO - DataFrame cleaned!


In [11]:
df_load_energy_one_column.index = pd.to_datetime(df_load_energy_one_column.index, format="mixed", yearfirst=True)

## Transforming multi index columns into one column name with the Region names

In [12]:
df_load_energy_one_column.columns = df_load_energy_one_column.columns.droplevel(list(range(0, 2)))

In [13]:
df_load_energy_one_column.head()

nom_subsistema,Norte,Nordeste,Sul,Sudeste/Centro-Oeste
din_instante,,,,
2000-01-01,2243.512500,4646.370833,4800.650000,19045.995833
2000-01-02,2259.808333,4895.720833,4899.800000,19398.025000
2000-01-03,2407.429167,5736.008333,6261.554167,23061.745833
2000-01-04,2404.216667,5783.770833,6733.741667,24228.241667
2000-01-05,2439.650000,5753.120833,6961.170833,24807.937500


## Creating an output path to save the dataframe after preprocessing

In [14]:
from utils.dataframe_tools import save_csv

In [15]:
save_csv(df_load_energy_one_column, path_to_save=OUTPUT_PATH, filename=f"energy_load_curve_brazil")

2023-05-17 11:54:40,851 - root - INFO - Dataframe outputs/energy_load_curve_brazil.csv saved!
